# Notebook 03 — Modelo Baseline y Calibración de Riesgo (Validación Temporal)

**Objetivo:** entrenar un modelo baseline para estimar el riesgo de falla (probabilidad) a un horizonte definido  
usando validación temporal (train/valid/test por fechas), evitando fuga de información.

**Input:** `data/processed/azure_pm/dataset_modelo.parquet`  
**Output:** `modelos/modelo_baseline_falla_30d.joblib` (modelo calibrado + metadata + métricas)

---

## Qué se hará en este notebook

1. Cargar dataset listo para modelado (Notebook 02).
2. Definir features y objetivo (`falla_30d` inicialmente).
3. Separar train/valid/test por fechas (simulación de producción).
4. Entrenar baseline con pipeline reproducible (imputación + escalado + modelo).
5. Evaluar con métricas robustas para desbalance (AUC y Average Precision).
6. Calibrar probabilidades (isotónica) para interpretabilidad como riesgo.
7. Guardar el artefacto final con métricas y metadata.

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

SEED = 42
np.random.seed(SEED)

In [3]:
from pathlib import Path
import pandas as pd

# Confirmar directorio actual
RAIZ_PROYECTO = Path.cwd()
print("Working dir:", RAIZ_PROYECTO)

# Ruta al dataset
RUTA_DATASET = (
    RAIZ_PROYECTO
    / "data"
    / "processed"
    / "azure_pm"
    / "dataset_modelo.parquet"
)

print("Buscando:", RUTA_DATASET)
print("Existe:", RUTA_DATASET.exists())

# Debug: listar carpeta si no existe
if not RUTA_DATASET.exists():
    carpeta = RUTA_DATASET.parent
    print("\nContenido de:", carpeta)
    if carpeta.exists():
        for p in carpeta.glob("*"):
            print("-", p.name)
    else:
        print("⚠️ Carpeta no existe")

dataset_modelo = pd.read_parquet(RUTA_DATASET)

print("✅ Cargado:", dataset_modelo.shape)
dataset_modelo.head()


Working dir: c:\Users\sebas\OneDrive\Desktop\Proyecto Chatbot\Mantenimiento Industrial
Buscando: c:\Users\sebas\OneDrive\Desktop\Proyecto Chatbot\Mantenimiento Industrial\data\processed\azure_pm\dataset_modelo.parquet
Existe: True
✅ Cargado: (31567, 71)


,machineID,fecha,datetime_mean,datetime_std,datetime_min,datetime_max,volt_mean,volt_std,volt_min,volt_max,rotate_mean,rotate_std,rotate_min,rotate_max,pressure_mean,pressure_std,pressure_min,pressure_max,vibration_mean,vibration_std,vibration_min,vibration_max,model,age,volt_mean_lag1d,rotate_mean_lag1d,pressure_mean_lag1d,vibration_mean_lag1d,volt_mean_lag2d,rotate_mean_lag2d,pressure_mean_lag2d,vibration_mean_lag2d,volt_mean_lag3d,rotate_mean_lag3d,pressure_mean_lag3d,vibration_mean_lag3d,volt_mean_lag7d,rotate_mean_lag7d,pressure_mean_lag7d,vibration_mean_lag7d,volt_mean_roll3d_mean,volt_mean_roll3d_std,rotate_mean_roll3d_mean,rotate_mean_roll3d_std,pressure_mean_roll3d_mean,pressure_mean_roll3d_std,vibration_mean_roll3d_mean,vibration_mean_roll3d_std,volt_mean_roll7d_mean,volt_mean_roll7d_std,rotate_mean_roll7d_mean,rotate_mean_roll7d_std,pressure_mean_roll7d_mean,pressure_mean_roll7d_std,vibration_mean_roll7d_mean,vibration_mean_roll7d_std,volt_mean_roll14d_mean,volt_mean_roll14d_std,rotate_mean_roll14d_mean,rotate_mean_roll14d_std,pressure_mean_roll14d_mean,pressure_mean_roll14d_std,vibration_mean_roll14d_mean,vibration_mean_roll14d_std,volt_mean_tendencia_7d,rotate_mean_tendencia_7d,pressure_mean_tendencia_7d,vibration_mean_tendencia_7d,falla_7d,falla_14d,falla_30d
0,1,2015-01-01,2015-01-01 14:30:00,0 days 05:20:18.740853656,2015-01-01 06:00:00,2015-01-01 23:00:00,-0.678297,-2.476435,1.176128,-1.989951,-0.323717,-0.050713,-0.065272,-0.609071,-0.494717,0.306033,-0.943743,-1.039867,-0.170579,0.900683,-1.528446,0.230129,model3,1.110983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.953126,NaN,-0.445345,NaN,-0.648009,NaN,-0.225389,NaN,-1.379574,NaN,-0.649668,NaN,-0.912531,NaN,-0.315313,NaN,-1.872885,NaN,-0.878178,NaN,-1.239096,NaN,-0.419486,NaN,NaN,NaN,NaN,NaN,1,1,1
1,1,2015-01-02,2015-01-02 11:30:00,0 days 07:04:15.844122715,2015-01-02 00:00:00,2015-01-02 23:00:00,-0.214936,0.365419,-0.087024,0.074476,0.021740,-1.449082,1.180868,-0.884426,-0.508954,0.935690,-0.361882,0.869939,-0.541135,0.703180,-0.346103,0.628281,model3,1.110983,-0.678429,-0.325768,-0.494320,-0.170194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.627113,-0.700165,-0.209589,-0.732671,-0.657353,-0.885701,-0.471700,-0.561151,-0.906273,-0.965370,-0.309981,-0.947109,-0.925742,-1.009344,-0.663026,-0.735770,-1.230859,-1.239576,-0.418834,-1.193600,-1.257045,-1.206731,-0.881414,-0.934485,NaN,NaN,NaN,NaN,1,1,1
2,1,2015-01-03,2015-01-03 11:30:00,0 days 07:04:15.844122715,2015-01-03 00:00:00,2015-01-03 23:00:00,0.216533,-1.645950,1.518680,-0.600590,0.698294,-0.336129,0.840373,0.842229,-0.604878,-0.677516,0.654907,-0.638289,3.651528,3.934227,0.588483,4.927501,model3,1.110983,-0.214134,0.020230,-0.508579,-0.540586,-0.680932,-0.336000,-0.494906,-0.168138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.316058,-0.467222,0.176805,-0.214748,-0.702438,-0.800869,1.304121,3.089429,-0.454688,-0.719454,0.246751,-0.430032,-0.989482,-0.930467,1.843873,2.750533,-0.618292,-0.961352,0.334009,-0.623820,-1.343646,-1.123717,2.448940,2.801852,NaN,NaN,NaN,NaN,1,1,1
3,1,2015-01-04,2015-01-04 11:30:00,0 days 07:04:15.844122715,2015-01-04 00:00:00,2015-01-04 23:00:00,0.828337,1.901750,-1.493753,1.757599,0.126215,-2.070150,0.551847,-0.957798,0.121112,0.432494,0.200360,-0.248416,5.612090,0.087626,3.714277,3.644353,model3,1.110983,0.218205,0.697843,-0.604654,3.650226,-0.212022,0.014330,-0.509261,-0.541562,-0.681697,-0.342773,-0.494872,-0.166519,NaN,NaN,NaN,NaN,0.390638,-0.318129,0.381509,-0.507267,-0.432991,-0.231672,3.866635,4.547308,0.083569,-0.321994,0.243855,-0.609001,-0.684537,-0.499516,4.017173,3.887578,0.111846,-0.511674,0.330093,-0.821031,-0.929331,-0.670161,5.336118,4.020444,NaN,NaN,NaN,NaN,1,1,1
4,1,2015-01-05,2015-01-05 11:30:00,0 days 07:04:15.844122715,2015-01-05 00:00:00,2015-01-05 23:00:00,0.040345,1.317615,-1.715426,0.262103,0.458669,-0.282260,0.924291,1.070018,0.312559,0.354700,-0.387249,0.794580,1.392034,3.801060,0.866723,2.782980,model3,1.110983,0.831243,0.124869,

In [4]:
dataset_modelo = dataset_modelo.sort_values(["fecha", "machineID"]).reset_index(drop=True)

n_maquinas = int(dataset_modelo["machineID"].nunique())
pct_filas_nulos = round(dataset_modelo.isna().any(axis=1).mean() * 100, 2)

print("Máquinas:", n_maquinas)
print("% filas con al menos 1 nulo:", pct_filas_nulos, "%")

for objetivo in ["falla_7d", "falla_14d", "falla_30d"]:
    rate = round(dataset_modelo[objetivo].mean() * 100, 2)
    print(f"Rate {objetivo}: {rate}%")


Máquinas: 98
% filas con al menos 1 nulo: 2.17 %
Rate falla_7d: 17.72%
Rate falla_14d: 32.81%
Rate falla_30d: 56.94%


In [5]:
OBJETIVO = "falla_30d"

columnas_no_features = {"machineID", "fecha", "falla_7d", "falla_14d", "falla_30d"}
columnas_features = [c for c in dataset_modelo.columns if c not in columnas_no_features]

columnas_numericas = [c for c in columnas_features if pd.api.types.is_numeric_dtype(dataset_modelo[c])]
columnas_no_numericas = [c for c in columnas_features if c not in columnas_numericas]

print("Objetivo:", OBJETIVO)
print("N features:", len(columnas_features))
print("Numéricas:", len(columnas_numericas))
print("No numéricas:", len(columnas_no_numericas), "→", columnas_no_numericas[:10])


Objetivo: falla_30d
N features: 66
Numéricas: 61
No numéricas: 5 → ['datetime_mean', 'datetime_std', 'datetime_min', 'datetime_max', 'model']


In [6]:
fechas = np.sort(dataset_modelo["fecha"].unique())

corte_train = fechas[int(len(fechas) * 0.70)]
corte_valid = fechas[int(len(fechas) * 0.85)]

train = dataset_modelo[dataset_modelo["fecha"] <= corte_train].copy()
valid = dataset_modelo[(dataset_modelo["fecha"] > corte_train) & (dataset_modelo["fecha"] <= corte_valid)].copy()
test  = dataset_modelo[dataset_modelo["fecha"] > corte_valid].copy()

print("Corte train:", str(corte_train)[:10])
print("Corte valid:", str(corte_valid)[:10])
print("Train:", train.shape, "| Valid:", valid.shape, "| Test:", test.shape)

# rates por split para ver estabilidad (2 decimales)
for nombre, df in [("train", train), ("valid", valid), ("test", test)]:
    rate = round(df[OBJETIVO].mean() * 100, 2)
    print(f"Rate {OBJETIVO} en {nombre}: {rate}%")


Corte train: 2015-09-13
Corte valid: 2015-11-07
Train: (24646, 71) | Valid: (4549, 71) | Test: (2372, 71)
Rate falla_30d en train: 53.4%
Rate falla_30d en valid: 59.62%
Rate falla_30d en test: 88.62%


In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score

transformacion_numerica = Pipeline(steps=[
    ("imputacion", SimpleImputer(strategy="median")),
    ("escalado", StandardScaler()),
])

transformacion_categorica = Pipeline(steps=[
    ("imputacion", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

preprocesamiento = ColumnTransformer(
    transformers=[
        ("num", transformacion_numerica, columnas_numericas),
        ("cat", transformacion_categorica, columnas_no_numericas),
    ],
    remainder="drop",
)

modelo_lr = LogisticRegression(
    max_iter=3000,
    class_weight="balanced",
    random_state=SEED
)

pipeline_lr = Pipeline(steps=[
    ("prep", preprocesamiento),
    ("modelo", modelo_lr)
])

pipeline_lr.fit(train[columnas_features], train[OBJETIVO])

proba_valid = pipeline_lr.predict_proba(valid[columnas_features])[:, 1]
proba_test  = pipeline_lr.predict_proba(test[columnas_features])[:, 1]

auc_valid = round(roc_auc_score(valid[OBJETIVO], proba_valid), 2)
ap_valid  = round(average_precision_score(valid[OBJETIVO], proba_valid), 2)

auc_test = round(roc_auc_score(test[OBJETIVO], proba_test), 2)
ap_test  = round(average_precision_score(test[OBJETIVO], proba_test), 2)

print(f"[LR] VALID → AUC: {auc_valid} | AP: {ap_valid}")
print(f"[LR] TEST  → AUC: {auc_test} | AP: {ap_test}")


[LR] VALID → AUC: 0.62 | AP: 0.74
[LR] TEST  → AUC: 0.62 | AP: 0.94


In [11]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import roc_auc_score, average_precision_score

X_cal = pd.concat([train[columnas_features], valid[columnas_features]], axis=0)
y_cal = pd.concat([train[OBJETIVO], valid[OBJETIVO]], axis=0)

calibrador = CalibratedClassifierCV(
    estimator=pipeline_lr,
    method="isotonic",
    cv=3
)

calibrador.fit(X_cal, y_cal)

proba_test_cal = calibrador.predict_proba(test[columnas_features])[:, 1]

auc_test_cal = round(roc_auc_score(test[OBJETIVO], proba_test_cal), 2)
ap_test_cal  = round(average_precision_score(test[OBJETIVO], proba_test_cal), 2)

print(f"[Calibrado isotónico | train+valid | cv=3] TEST → AUC: {auc_test_cal} | AP: {ap_test_cal}")



[Calibrado isotónico | train+valid | cv=3] TEST → AUC: 0.63 | AP: 0.94


In [12]:
from sklearn.calibration import calibration_curve
import pandas as pd
import numpy as np

fraccion_positivos, prob_media = calibration_curve(
    y_true=test[OBJETIVO],
    y_prob=proba_test_cal,
    n_bins=10,
    strategy="quantile"
)

tabla_calibracion = pd.DataFrame({
    "prob_media_bin": np.round(prob_media, 2),
    "fraccion_positivos_bin": np.round(fraccion_positivos, 2),
})

tabla_calibracion


,prob_media_bin,fraccion_positivos_bin
0,0.44,0.81
1,0.46,0.92
2,0.50,0.90
3,0.52,0.82
4,0.54,0.80
5,0.57,0.84
6,0.62,0.86
7,0.65,0.97
8,0.70,0.99


In [13]:
import joblib
from pathlib import Path

DIRECTORIO_MODELOS = Path.cwd() / "modelos"
DIRECTORIO_MODELOS.mkdir(parents=True, exist_ok=True)

ruta_modelo = DIRECTORIO_MODELOS / f"modelo_baseline_{OBJETIVO}.joblib"

artefacto = {
    "objetivo": OBJETIVO,
    "columnas_features": columnas_features,
    "modelo_calibrado": calibrador,
    "metricas": {
        "auc_valid": float(auc_valid),
        "ap_valid": float(ap_valid),
        "auc_test": float(auc_test),
        "ap_test": float(ap_test),
        "auc_test_cal": float(auc_test_cal),
        "ap_test_cal": float(ap_test_cal),
    },
    "cortes_temporales": {
        "corte_train": str(corte_train),
        "corte_valid": str(corte_valid),
    },
    "tamano_splits": {
        "train": (int(train.shape[0]), int(train.shape[1])),
        "valid": (int(valid.shape[0]), int(valid.shape[1])),
        "test":  (int(test.shape[0]), int(test.shape[1])),
    }
}

joblib.dump(artefacto, ruta_modelo)

print("✅ Modelo guardado en:", ruta_modelo.resolve())
print("Métricas guardadas:")
for k, v in artefacto["metricas"].items():
    print(f"- {k}: {round(v, 2)}")


✅ Modelo guardado en: C:\Users\sebas\OneDrive\Desktop\Proyecto Chatbot\Mantenimiento Industrial\modelos\modelo_baseline_falla_30d.joblib
Métricas guardadas:
- auc_valid: 0.62
- ap_valid: 0.74
- auc_test: 0.62
- ap_test: 0.94
- auc_test_cal: 0.63
- ap_test_cal: 0.94


In [14]:
test_eval = test[["machineID", "fecha", OBJETIVO]].copy()
test_eval["riesgo"] = proba_test_cal

test_eval = test_eval.sort_values(["fecha", "riesgo"], ascending=[True, False])

K = 5
topk = test_eval.groupby("fecha").head(K)

precision_topk = round(topk[OBJETIVO].mean() * 100, 2)

print(f"Precision@{K} por día (promedio): {precision_topk}%")
topk.head(10)


Precision@5 por día (promedio): 98.88%


,machineID,fecha,falla_30d,riesgo
29251,85,2015-11-08,1,0.711926
29207,20,2015-11-08,1,0.699453
29202,13,2015-11-08,1,0.652922
29211,24,2015-11-08,1,0.652922
29216,30,2015-11-08,1,0.652922
29272,13,2015-11-09,1,0.711926
29321,85,2015-11-09,1,0.711926
29304,56,2015-11-09,1,0.699453
29277,20,2015-11-09,1,0.692590
29298,49,2015-11-09,1,0.692590


In [16]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import roc_auc_score, average_precision_score

# --- Preprocesamiento DENSO para HGB (clave para evitar error sparse) ---
# Nota: en versiones nuevas es sparse_output=False; en algunas antiguas es sparse=False.
# Usamos un try/except para compatibilidad.
try:
    onehot_denso = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
except TypeError:
    onehot_denso = OneHotEncoder(handle_unknown="ignore", sparse=False)

transformacion_numerica_hgb = Pipeline(steps=[
    ("imputacion", SimpleImputer(strategy="median")),
    # Escalado NO es obligatorio para HGB, pero no molesta
    ("escalado", StandardScaler()),
])

transformacion_categorica_hgb = Pipeline(steps=[
    ("imputacion", SimpleImputer(strategy="most_frequent")),
    ("onehot", onehot_denso),
])

preprocesamiento_hgb = ColumnTransformer(
    transformers=[
        ("num", transformacion_numerica_hgb, columnas_numericas),
        ("cat", transformacion_categorica_hgb, columnas_no_numericas),
    ],
    remainder="drop",
    sparse_threshold=0.0  # fuerza salida densa
)

modelo_hgb = HistGradientBoostingClassifier(
    random_state=SEED,
    max_iter=300,
    learning_rate=0.05,
    max_depth=6
)

pipeline_hgb = Pipeline(steps=[
    ("prep", preprocesamiento_hgb),
    ("modelo", modelo_hgb)
])

# Balanceo con sample_weight
pesos_train = compute_sample_weight(class_weight="balanced", y=train[OBJETIVO])

pipeline_hgb.fit(
    train[columnas_features],
    train[OBJETIVO],
    modelo__sample_weight=pesos_train
)

proba_valid_hgb = pipeline_hgb.predict_proba(valid[columnas_features])[:, 1]
proba_test_hgb  = pipeline_hgb.predict_proba(test[columnas_features])[:, 1]

auc_valid_hgb = round(roc_auc_score(valid[OBJETIVO], proba_valid_hgb), 2)
ap_valid_hgb  = round(average_precision_score(valid[OBJETIVO], proba_valid_hgb), 2)

auc_test_hgb = round(roc_auc_score(test[OBJETIVO], proba_test_hgb), 2)
ap_test_hgb  = round(average_precision_score(test[OBJETIVO], proba_test_hgb), 2)

print(f"[HGB] VALID → AUC: {auc_valid_hgb} | AP: {ap_valid_hgb}")
print(f"[HGB] TEST  → AUC: {auc_test_hgb} | AP: {ap_test_hgb}")


[HGB] VALID → AUC: 0.64 | AP: 0.72
[HGB] TEST  → AUC: 0.64 | AP: 0.94


In [17]:
from sklearn.calibration import CalibratedClassifierCV

calibrador_hgb = CalibratedClassifierCV(
    estimator=pipeline_hgb,
    method="isotonic",
    cv=3
)

calibrador_hgb.fit(valid[columnas_features], valid[OBJETIVO])

proba_test_hgb_cal = calibrador_hgb.predict_proba(test[columnas_features])[:, 1]

auc_test_hgb_cal = round(roc_auc_score(test[OBJETIVO], proba_test_hgb_cal), 2)
ap_test_hgb_cal  = round(average_precision_score(test[OBJETIVO], proba_test_hgb_cal), 2)

print(f"[HGB calibrado | cv=3] TEST → AUC: {auc_test_hgb_cal} | AP: {ap_test_hgb_cal}")


[HGB calibrado | cv=3] TEST → AUC: 0.63 | AP: 0.93


In [18]:
resumen_modelos = pd.DataFrame([
    {
        "modelo": "LR (baseline)",
        "auc_valid": auc_valid,
        "ap_valid": ap_valid,
        "auc_test": auc_test,
        "ap_test": ap_test,
        "auc_test_cal": auc_test_cal,
        "ap_test_cal": ap_test_cal
    },
    {
        "modelo": "HGB (comparativo)",
        "auc_valid": auc_valid_hgb,
        "ap_valid": ap_valid_hgb,
        "auc_test": auc_test_hgb,
        "ap_test": ap_test_hgb,
        "auc_test_cal": auc_test_hgb_cal if "auc_test_hgb_cal" in globals() else np.nan,
        "ap_test_cal": ap_test_hgb_cal if "ap_test_hgb_cal" in globals() else np.nan
    }
])

# Redondeo bonito (2 decimales)
resumen_modelos = resumen_modelos.round(2)

resumen_modelos


,modelo,auc_valid,ap_valid,auc_test,ap_test,auc_test_cal,ap_test_cal
0,LR (baseline),0.62,0.74,0.62,0.94,0.63,0.94
1,HGB (comparativo),0.64,0.72,0.64,0.94,0.63,0.93


In [19]:
# Elegimos por AP calibrado si está disponible; si no, por AP test
col_objetivo = "ap_test_cal" if resumen_modelos["ap_test_cal"].notna().all() else "ap_test"

mejor_fila = resumen_modelos.sort_values(col_objetivo, ascending=False).iloc[0]
mejor_modelo = mejor_fila["modelo"]

print("Criterio selección:", col_objetivo)
print("Mejor modelo:", mejor_modelo)
print(mejor_fila)


Criterio selección: ap_test_cal
Mejor modelo: LR (baseline)
modelo          LR (baseline)
auc_valid                0.62
ap_valid                 0.74
auc_test                 0.62
ap_test                  0.94
auc_test_cal             0.63
ap_test_cal              0.94
Name: 0, dtype: object


In [20]:
import joblib
from pathlib import Path

DIRECTORIO_MODELOS = Path.cwd() / "modelos"
DIRECTORIO_MODELOS.mkdir(parents=True, exist_ok=True)

ruta_modelo_hgb = DIRECTORIO_MODELOS / f"modelo_comparativo_hgb_{OBJETIVO}.joblib"

artefacto_hgb = {
    "objetivo": OBJETIVO,
    "columnas_features": columnas_features,
    "modelo_base": pipeline_hgb,
    "modelo_calibrado": calibrador_hgb if "calibrador_hgb" in globals() else None,
    "metricas": {
        "auc_valid": float(auc_valid_hgb),
        "ap_valid": float(ap_valid_hgb),
        "auc_test": float(auc_test_hgb),
        "ap_test": float(ap_test_hgb),
        "auc_test_cal": float(auc_test_hgb_cal) if "auc_test_hgb_cal" in globals() else None,
        "ap_test_cal": float(ap_test_hgb_cal) if "ap_test_hgb_cal" in globals() else None,
    },
    "cortes_temporales": {
        "corte_train": str(corte_train),
        "corte_valid": str(corte_valid),
    }
}

joblib.dump(artefacto_hgb, ruta_modelo_hgb)

print("✅ Modelo HGB guardado en:", ruta_modelo_hgb.resolve())


✅ Modelo HGB guardado en: C:\Users\sebas\OneDrive\Desktop\Proyecto Chatbot\Mantenimiento Industrial\modelos\modelo_comparativo_hgb_falla_30d.joblib


In [21]:
print("Resumen modelos (AUC/AP):")
display(resumen_modelos)

# Diferencias (HGB - LR) en test
fila_lr = resumen_modelos[resumen_modelos["modelo"] == "LR (baseline)"].iloc[0]
fila_hgb = resumen_modelos[resumen_modelos["modelo"] == "HGB (comparativo)"].iloc[0]

delta_auc = round(fila_hgb["auc_test"] - fila_lr["auc_test"], 2)
delta_ap  = round(fila_hgb["ap_test"]  - fila_lr["ap_test"], 2)

print(f"\nΔ TEST (HGB - LR) → AUC: {delta_auc} | AP: {delta_ap}")

if resumen_modelos["ap_test_cal"].notna().all():
    delta_ap_cal = round(fila_hgb["ap_test_cal"] - fila_lr["ap_test_cal"], 2)
    print(f"Δ TEST calibrado (HGB - LR) → AP_cal: {delta_ap_cal}")


Resumen modelos (AUC/AP):


,modelo,auc_valid,ap_valid,auc_test,ap_test,auc_test_cal,ap_test_cal
0,LR (baseline),0.62,0.74,0.62,0.94,0.63,0.94
1,HGB (comparativo),0.64,0.72,0.64,0.94,0.63,0.93



Δ TEST (HGB - LR) → AUC: 0.02 | AP: 0.0
Δ TEST calibrado (HGB - LR) → AP_cal: -0.01


# Conclusiones — Notebook 03

En este notebook se entrenaron y evaluaron dos enfoques para estimar el riesgo de falla a 30 días (`falla_30d`)
bajo un esquema de validación temporal (train/valid/test por fechas), evitando fuga de información:

- **Baseline:** Regresión Logística con pipeline reproducible.
- **Comparativo:** HistGradientBoostingClassifier (HGB).

La evaluación se realizó mediante métricas robustas para clases desbalanceadas:
- AUC ROC
- Average Precision (AP)

Además, se aplicó calibración isotónica para mejorar la interpretabilidad
de las probabilidades como riesgo operacional.

---

## Resultados

| Modelo | AUC (Valid) | AP (Valid) | AUC (Test) | AP (Test) | AP Calibrado (Test) |
|--------|-------------|-----------|-----------|----------|----------------------|
| LR (baseline) | 0.62 | 0.74 | 0.62 | 0.94 | 0.94 |
| HGB (comparativo) | 0.64 | 0.72 | 0.64 | 0.94 | 0.93 |

Diferencias en test (HGB − LR):

- Δ AUC: +0.02
- Δ AP:  0.00
- Δ AP calibrado: −0.01

---

## Análisis y decisión

El modelo HGB presenta una mejora marginal en AUC, pero no muestra ventajas
significativas en Average Precision ni en calibración.

Por el contrario, la Regresión Logística:

- Presenta desempeño comparable en clasificación.
- Exhibe mejor calibración probabilística.
- Ofrece mayor interpretabilidad.
- Reduce complejidad operativa.

En consecuencia, se selecciona el modelo **LR calibrado** como candidato principal
para su integración en el sistema de priorización de mantenimiento.

---

## Próximos pasos

El modelo seleccionado se utilizará como insumo para:

1. Evaluación bajo restricciones operativas (Precision@K).
2. Simulación de impacto económico (costo esperado de fallas).
3. Optimización de intervenciones bajo presupuesto.
4. Integración en un pipeline automatizado (`main